# 0. Simulation code

In [1]:
import numpy as np
import random
import sympy 
import scipy.stats as st
import scipy.special
import pandas
import csv
import math
from decimal import *
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
import datetime
from tqdm import tqdm

In [2]:
class switch(object):
    def __init__(self, value):
        self.value = value
        self.fall = False
    def __iter__(self):
        """Return the match method once, then stop"""
        yield self.match
        raise StopIteration  
    def match(self, *args):
        """Indicate whether or not to enter a case suite"""
        if self.fall or not args:
            return True
        elif self.value in args: # changed for v1.5, see below
            self.fall = True
            return True
        else:
            return False

In [3]:
def main(sample,N,N_e):
    # define parameter
    cobject = init_UORAparameter("UORA", sample, N, N_e)
    
    # sample loop
    for ite in range(0, cobject.sample):                       
        # initail parameter
        cobject.define_RAparameter()         
#         print('ite:',ite) # debug # DEBUG
        # slot loop
        for slot in range(0, len(cobject.slot_list)):
            # arrival process
#             print('slot:',slot) # debug # DEBUG
            cobject.arrival_process(slot)
    
            if cobject.nUE <=90:

#                 print("Andy ",len(cobject.dict_retr))
                if len(cobject.dict_retr) == 0:
                    if slot >= cobject.iteImax:
                        cobject.iteImax = slot #record Imax # slot+1 because "slot" start form 0, but real slot start from 1 
                        break

            else:               

#                 print("Andy ",len(cobject.dict_retr))
                if len(cobject.dict_retr) == 0:
                    
                    if slot >= cobject.iteImax:
                        cobject.iteImax = slot #record Imax # slot+1 because "slot" start form 0, but real slot start from 1 
                        

            # departure process 
            if len(cobject.dict_retr) > 0:
                cobject.departure_process(slot, cobject.reserveRARU[slot]) 
            else:
#                 cobject.iteImax = slot + 1 #record Imax # slot+1 because "slot" start form 0, but real slot start from 1 
                break  
        
        # accumulate statistic value from slot 0 to Imax
        cobject.calculate_statistic()

    # return vaule (success probability)
    if cobject.nUE >= 0:
        cobject.report()
#         return cobject.nUE, cobject.convRA.nUE, cobject.MsT, cobject.MS, cobject.convRA.MS, cobject.P_s_T, cobject.P_s, cobject.convRA.P_s ,  cobject.DaT, cobject.Da, cobject.convRA.Da , cobject.TpT, cobject.Tp, cobject.convRA.Tp, cobject.argImax, cobject.convRA.argImax 
        return cobject.nUE, cobject.MS, cobject.P_s ,cobject.Da, cobject.Tp, cobject.argImax    

    else:
        return -1

In [4]:
# define class to inital RA parameter
class init_RAparameter():
    # initail parameter for statistics value
    def __init__(self, config, sample, N): 
#         print("RA. init, Ru=",N) # DEBUG
        self.config = config 
        self.sample = sample
        self.nUE = 0 # num_UE[0]# bug! ,
        self.subcarrier = N # Ru_i change over time
#         self.Rufix = N      # Ru   keep the same 
        self.MS = 0
        self.P_s =  0
        self.iteMiSn = [ [0] * (retrans + 1) for _ in range(0, maxslot())]
        self.iteMiFn = [ [0] * (retrans + 1) for _ in range(0, maxslot())]
        self.Da = 0
        self.Da_MiSn = 0
        self.Dd = 0
        self.Dd_MiSn = 0
        self.L = {}
        self.iteN_RU = [ 0 for j in range(0, maxslot())]
        self.T_DATA = 0
        self.reserveRARU = [0 for _ in range(0,maxslot())] ##UORA 
        self.iteImax =0  ## UORA
        self.argImax = 0 ## UORA 

        self.Tp = 0      ## UORA
    # initail parameter for each iteration
    def define_RAparameter(self):
#         print("RA. defineRA") # DEBUG
##      self.dict_retr = 0 ＃因為格式應該為字典而非int
        # initial dict_prea = {0:0, 1:0, ..., nUE:0}, UE index : preamble
        self.dict_prea = gen_UE_prea(self.nUE)
        # initial slot_lit = [[], [], [], ..., []], ue number in each slot
        self.slot_list = [ [] for j in range(0, maxslot())]
        # initial dict_backoffcnt = {0:0, 1:0, ..., nUE:0}, UE index : backoff counter
        self.dict_backoffcnt = gen_UE_backoffcnt(self.nUE)
        # initial UE_list[UE_id][x]
        # x=0 arrive time,     x=1 1st attempt time, x=2 2nd attemp time
        # x=3 3nd attemp time, x=4 4nd attempt time, x=5 RA status
        self.UE_list = [ [-1] * (retrans + 4) for _ in range(0, self.nUE)] # retrans +3 for Ra /retrans +4 for UORA
        # initail arrive_num from slot0 to Imax
        self.arrive_num = [ 0 for j in range(0, maxslot())]
        # inital statistic value for each iteration
        self.MiSn = [ [0] * (retrans + 1) for _ in range(0, maxslot())]
        self.MiFn = [ [0] *( retrans + 1) for _ in range(0, maxslot())]
        self.T = [ 0 for j in range(0, maxslot())]
        self.N_RU = [ 0 for j in range(0, maxslot())]
    # arrival process:
    def arrival_process(self, slot):
#         print("RA. arrival") # DEBUG
        # one-shot
        if slot == 0 :
            if(CONFIG=='UORA'):
                # initial dict_retr = {0:0, 1:0, ..., nUE:0}, UE index : retransmission counter
                self.dict_retr = gen_UE_id(self.nUE)          # 設定傳送次數
                self.arrive_num[slot] = len(self.dict_retr)   # 紀錄slot arrival STA數量
                for i in range (0,self.nUE):
                    init_obo = int(random.uniform(0,UORA_BO[0]+1))# update initOBO
                    self.dict_backoffcnt[i] = init_obo            # 設定OBO
            else:
                # initial dict_retr = {0:0, 1:0, ..., nUE:0}, UE index : retransmission counter
                self.dict_retr = gen_UE_id(self.nUE)
                self.arrive_num[slot] = len(self.dict_retr) #else1
                for i in range (0, self.nUE):
                    self.dict_backoffcnt[i] = 1
        else:
            self.arrive_num[slot] = len(self.dict_retr) #else2
    # departure process:
    def departure_process(self, slot, countdown):
#         print("RA.  departure, Countdown=",countdown) # DEBUG
        for ue in list(self.dict_retr):
            self.dict_backoffcnt[ue] -= countdown
            # UORA 
            self.subcarrier = countdown # 令subcarrier 為RU數目，for doRA 
            # when backoff conunter equal to 0 means it's time to contend the RACH resource
            if self.dict_backoffcnt[ue] <= 0:
                # genearate preamble for each of ue, EX: dict_prea = {0:23, 1:34, ..., nUE:16}
                self.dict_prea[ue] = doRA(self.subcarrier)  
                # append ue in slot_list to know which ue in current slot, EX: slot_list = [[0,1,2,...,nUE], [...], ..., [...]]
                self.slot_list[slot].append(ue)
                # record infomation in UE_list
                self.UE_list[ue][self.dict_retr[ue] + 1] = slot
        # check the preamble to get the success ue and remove the success ue from dict_retr 
        self.check_preamble(slot, self.subcarrier)
        # add retranmsission counter for failure ue in dict_retr
        mark_retrans(self.dict_retr, self.slot_list[slot], self.UE_list)
        # give the random backoff counter for the ue in dict_retr  
        backoff(self.dict_retr, self.dict_backoffcnt, self.slot_list, slot)
    # check preamble:
    def check_preamble(self, slot, subcarrier):
        # check preamble from subcarrier 1 to N
        for preamble in range(1, subcarrier + 1):
            # get which ue choose current preamble, EX: collided_ue = [1, 5, 23...]
            collided_ue = map_preamble(self.dict_prea, self.slot_list[slot], preamble) 
            # make sure the collided_ue is larger than 0 that means at least one ue choose this premable
            # and mark this ue is success
            if len(collided_ue) != 0:
                get_succuss_ue(self, slot, collided_ue)
    # accumulate the MiSn, MiFn and N_RU for each of iteration
    def calculate_statistic(self):
        # Accumulate MiSn, MiFn, N_RU
        for temp_i in range(0, maxslot()):
            self.iteN_RU[temp_i] += self.N_RU[temp_i]
            for temp_n in range (0, retrans + 1):
                self.iteMiSn[temp_i][temp_n] += self.MiSn[temp_i][temp_n] 
                self.iteMiFn[temp_i][temp_n] += self.MiFn[temp_i][temp_n] 
    # calculate the average MiSn, MiFn and N_RU
    def get_MS(self):
        self.MS = 0
        self.argImax = 0
        self.argImax = self.iteImax #/ float(self.sample) ## UORA
        for temp_i in range(0, maxslot()):
            # Average total data size / accumulate data size in each slot
            self.iteN_RU[temp_i] = self.iteN_RU[temp_i] / float(self.sample)
            self.T_DATA += self.iteN_RU[temp_i]
            for temp_n in range (0,retrans+1):
                self.iteMiSn[temp_i][temp_n] = self.iteMiSn[temp_i][temp_n] / float(self.sample)
                self.iteMiFn[temp_i][temp_n] = self.iteMiFn[temp_i][temp_n] / float(self.sample)
                self.MS += self.iteMiSn[temp_i][temp_n]

        self.MS = round((self.MS) * 1000000) / 1000000
    # calculate the Ps, Da, Dd, T_DATA
    def report(self):
        self.get_MS()

        # Sucesss Probability - convRA
        if self.nUE != 0:
            self.P_s = round((float(self.MS) / float(self.nUE)) * 1000000) / 1000000
#             print('report,PsN:',self.P_s,'MS:',self.MS,'nUE:',self.nUE) # DEBUG
        else:
            self.P_s = 0.0
        
        # Utilization - convRA
        # self.Uti = 0
        # if self.subcarrier!=0:
        #     self.Uti = round((float(self.MS) / float(maxslot() * self.subcarrier)) * 1000000) / 1000000
        # else:
        #     self.Uti = 0.0

        # Access delay and Data delay - convRA
        for temp_i in range(0, maxslot()):
#             self.T[temp_i] = (temp_i) * T_RAREP + T_RAR + W_RAR + T_N_CR + T_ATCH + self.T_DATA
#             self.T[temp_i]=(temp_i)+K ## UORA 舊的
            for temp_n in range (0, retrans + 1):
                self.Da_MiSn += self.iteMiSn[temp_i][temp_n] * float( K*(temp_i+1+ith_slot) + ((K-1)*K)/2) # UORA 新Da
        if self.nUE != 0 and self.MS != 0:
            # self.Da = float(self.Da_MiSn) / float(self.MS)
#             self.Dd = float(self.Dd_MiSn) / float(self.MS)
            self.Da = float(self.Da_MiSn*5.681) #/ float(self.MS * K) ##UORA TXOP=5.681
            
        else:
            self.Da = 0.0
#             self.Dd = 0.0
        
        # Throughput 
        # success packet/argImax
        if self.argImax !=0 and self.MS != 0:
            self.Tp = float(self.MS)/float(self.argImax) ## UORA
        else:
            self.Tp = 0.0

In [5]:
# inherit the class of initializing RA parameter
class init_UORAparameter(init_RAparameter):
    def __init__(self, config, sample, N, N_e):
#         print('UORA. init') # DEBUG
        super().__init__(config, sample, N)             # RuA=N   建立物件
        self.convRA = init_RAparameter("UORA", sample, N_e) # RuN=N_e  建立物件
        self.totalsubcarrier = N + N_e # UORA 總共RU數目 ＝R
        self.const_RuA = N    #不變動的RuA,方便重新設定
        self.const_RuN = N_e  #不變動的RuN,方便重新設定
        self.accumulate_numUE = 0
        self.accumulate_numUE_EDT = 0
        self.P_s_T = 0
        self.MsT = 0
        self.Uti_T = 0
        self.DdT = 0
        self.TpT = 0
#     def grouping(self):
#         self.numUE_to_convRA = NumMn[0]
#         self.numUE_to_EDT = num_UE_EDT[0] - self.numUE_to_convRA
# #         self.dict_L_EDT = gen_UE_L(num_UE_EDT[0], L)
# #         self.numUE_to_convRA, self.numUE_to_EDT, self.convRA.L, self.L = compare_L_and_TBS(self.dict_L_EDT, num_UE_EDT[0])
# #         self.accumulate_numUE += (self.numUE_to_convRA + num_UE[0])
# #         self.accumulate_numUE_EDT += self.numUE_to_EDT
    def define_RAparameter(self):
#         self.grouping()
#         print("UORA.defineRA") # DEBUG
        self.nUE = NumMa[0] #self.numUE_to_EDT
#         print('UORA_defineRA,A self.const_RuA:',self.const_RuA) #對ite重設,不能用subcarrier(因為會變) # DEBUG
#         print('UORA_defineRA,N self.const_RuN:',self.const_RuN) # DEBUG
        self.reserveRARU = [self.const_RuA for _ in range(0,maxslot())]        # 紀錄associated預約的RU數目
        self.convRA.reserveRARU = [self.const_RuN for _ in range(0,maxslot())] # 紀錄Non-associated預約的RU數目
        super().define_RAparameter()
        self.convRA.nUE = NumM - NumMa[0] #self.num_UE_EDT[0]-self.numUE_to_EDT + num_UE[0] # 還不確定為什麼要加 + num_UE[0]
        self.convRA.define_RAparameter()
#         print('self.nUE:',self.nUE)
#         print('self.convRA.nUE:',self.convRA.nUE)
    def arrival_process(self, slot):
#         print("UORA. arrival") # DEBUG
        super().arrival_process(slot)
        self.convRA.arrival_process(slot)
    def departure_process(self, slot, countdown):
#         print("UORA. departure")# DEBUG
        super().departure_process(slot, countdown)
        if len(self.convRA.dict_retr) > 0:
#             self.convRA.departure_process(slot, self.totalsubcarrier - countdown) #nonassociated N-countdown
            self.convRA.departure_process(slot, self.const_RuN) #nonassociated N-countdown
    def calculate_statistic(self):
        super().calculate_statistic()
        self.convRA.calculate_statistic()
    def get_MS(self):
        super().get_MS()
    def report(self):
        # calculat the average ue number for EDT UE and non-EDT UE
#         self.nUE = self.accumulate_numUE_EDT / self.sample
#         self.convRA.nUE = self.accumulate_numUE / self.convRA.sample
        self.report_EDT()
        self.convRA.report()
        self.calculate_total()
    # calculate the Ps_E, Da_E, Dd_E, T_DATA_E
    def report_EDT(self):
        self.get_MS()
        # Sucesss Probability - EDT
        if self.nUE != 0:
            self.P_s = round((float(self.MS) / float(self.nUE)) * 1000000) / 1000000
        else:
            self.P_s = 0.0

        # Utilization - EDT
        # self.Uti = 0
        # if self.subcarrier != 0:
        #     self.Uti = round((float(self.MS) / float(maxslot() * self.subcarrier)) * 1000000) / 1000000
        # else:
        #     self.Uti = 0.0

        # Access delay - EDT
        for temp_i in range(0, maxslot()):
            for temp_n in range (0, retrans + 1):
                self.Da_MiSn += self.iteMiSn[temp_i][temp_n] * float( K*(temp_i+1+ith_slot) + ((K-1)*K)/2) # UORA Da 
        if self.nUE != 0 and self.MS != 0:
            self.Da = float(self.Da_MiSn*5.681) #/ float(self.MS * K) ##UORA TXOP=5.681
            print("Andy calculated da for grouping")
        else:
            self.Da = 0.0
        
        # Throughput
        ##UORA
        if self.argImax !=0 and self.MS !=0 :
            self.Tp = float(self.MS)/float(self.argImax)
        else:
            self.Tp = 0.0
    # calcualte the Ps_T, MsT, DdT, Uti_T
    def calculate_total(self):
        # PsT
        if self.nUE != 0 or self.convRA.nUE != 0:
            self.P_s_T = round(((float(self.MS) + float(self.convRA.MS)) / (float(self.nUE) + float(self.convRA.nUE))) * 1000000) / 1000000
        else:
            self.P_s_T = 0.0
        # MsT
        self.MsT = float(self.MS) + float(self.convRA.MS)
        # DaT
        if (self.MS * K + self.convRA.MS) != 0:
            self.DaT = round((self.Da * float(self.MS * K) + self.convRA.Da * float(self.convRA.MS)) / (float(self.MS * K) + float(self.convRA.MS)) * 1000000) / 1000000
        else:
            self.DaT = 0.0
        # TpT
        self.TpT = round((self.Tp + self.convRA.Tp)* 1000000) / 1000000
        
#         # DdT
#         if (self.MS + self.convRA.MS) != 0:
#             self.DdT = round((self.Dd * float(self.MS) + self.convRA.Dd * float(self.convRA.MS)) / (float(self.MS) + float(self.convRA.MS)) * 1000000) / 1000000
#         else:
#             self.DdT = 0.0
        # Uti
        # if self.subcarrier != 0 or self.convRA.subcarrier != 0:
        #     self.Uti_T = round(((float(self.MS) + float(self.convRA.MS)) / (float(maxslot() * (self.subcarrier + self.convRA.subcarrier)))) * 1000000) / 1000000
        # else:
        #     self.Uti_T = 0.0

In [6]:
# Give UE id for each UE
# dict_id = {0:0, 1:0, ..., nUE:0}
def gen_UE_id(num_UE):
    seq = [ i for i in range(0, num_UE)]
    dict_id = dict.fromkeys(seq, 0)
    return dict_id

# Initial dict_prea list and get the preamble of each of UE##
# dict_prea = {0:0, 1:0, ..., nUE:0}, UE index : preamble
def gen_UE_prea(num_UE):
    dict_prea = {}
    for i in range(0, num_UE):
        dict_prea[i] = 0
    return dict_prea

# Initial backoff counter for each of UE
# dict_backoffcnt = {0:0, 1:0, ..., nUE:0}, UE index : backoff counter
def gen_UE_backoffcnt(num_UE):
    dict_backoffcnt = {}
    for i in range(0, num_UE):
        dict_backoffcnt[i] = 0
    return dict_backoffcnt

# Calculate the maximum slot
def maxslot():
    # max slot can't be predict before run(calculate) it
    Maxraslot = raslot
#     Maxraslot = np.ceil((T_RAR + W_RAR + W_BO) / T_RAREP) * (retrans + 1) + 1  
    return int(Maxraslot)

# Give random packet size for each of ue
# dict_L = {0:35, 1:346, ..., nUE: 835}, UE index: packet length
def gen_UE_L(num_UE_EDT, L):                        
    dict_L = {}
    for i in range(0, num_UE_EDT):
        dict_L[i] = random.randint(1, L)
    return dict_L

# Compare the packet size with TBS and re-calculate the ue number for EDT UE and non-EDT UE
def compare_L_and_TBS(dict_L_EDT, num_UE_EDT):
    numUE2nonEDT = 0
    numUE2EDT = 0
    L_EDT = {}
    L_convRA = {}
    for ue in range(0, num_UE_EDT):
        if dict_L_EDT[ue] > TBS[0]:
            L_convRA[numUE2nonEDT] = dict_L_EDT[ue]
            numUE2nonEDT += 1  
        else:
            L_EDT[numUE2EDT] = dict_L_EDT[ue]
            numUE2EDT +=1
    
    #numUE2EDT = num_UE_EDT-numUE2nonEDT
    return numUE2nonEDT, num_UE_EDT-numUE2nonEDT, L_convRA, L_EDT

def normal(mean, std):
    u = random.random()
    v = random.random()
    return abs((np.sqrt(-2 * np.log(u)) * np.cos(2 * np.pi * v) * std) + mean)

# Give preamble for each of ue
def doRA(N):
    if N != 0:    
        return random.randint(1, N)
    else:
        return -1

# Check the preamble choosen by whihc ue
def map_preamble(dict_prea, ue_list, preamble):
    collided_ue = []
    # ue_list = slot_list[slot], ue_list is the ue in current slot
    for i in ue_list:
        # check each of ue's preamble(dict_prea) is equal to current preamble or not
        # and record which ue is same as current preamble
        if dict_prea[i] == preamble:
            collided_ue.append(i)
    return collided_ue

# Get the non collision ue and remove from the dict_retr
def get_succuss_ue(self, slot, collided_ue):
    # non collision
    if len(collided_ue) == 1 :
        success_ue = collided_ue[0]
        # get the retransmsission counter
        retry = self.dict_retr[success_ue]
        # add one success in MiSn[slot][retry]
        self.MiSn[slot][retry] += 1
        # mark ue status as '1'
#         self.UE_list[success_ue][5] = 1
        self.UE_list[success_ue][retrans+3] = 1 #[5] = 1
        # UORA reserve
        for k_temp in range(1,K):
#             print('slot+k_temp',slot+k_temp)
            # 預約之後k-1 slot
            self.reserveRARU[slot+k_temp] -= 1 #non-associated STA 不使用resrveRARU暫時不影響
        # accumulate the data size of successed UE in current slot
#         self.N_RU[slot] += self.L[success_ue]
        # remove success ue from dict_retr
        del self.dict_retr[success_ue]
    # more than one ue choose current preamble
    else:
        # loop all the collision ue
        for failed_ue in collided_ue:
            # get the retransmsission counter
            retry = self.dict_retr[failed_ue]
            # add one fialure in MiFn[slot][retry]
            self.MiFn[slot][retry] += 1
            # mark ue status as '0'
            self.UE_list[failed_ue][retrans+3] = 0
        success_ue = -1

# Add the retranmsission counter for the ue in dict_retr and current slot
# Delete the ue from the dict_retr if the ue reach to the maximum retransmission number
def mark_retrans(dict_retr, retr_ue, UE_list):
    # retr_ue = slot_list[slot], retr_ue is the ue in current slot
    for i in retr_ue:
        if i in dict_retr:
            # add retransmission counter for failure ue
            dict_retr[i] += 1
            # mark ue status as '2'
            UE_list[i][retrans + 3] = 2
            # check the reatransmission counter is larger than the max retranmsission times or not 
            if dict_retr[i] > retrans:
                # mark ue status as '0'
                UE_list[i][retrans + 3] = 0
                # exceed the max retransmission times, and remove this ue from dict_retr
                del dict_retr[i]

# Give the random backoff counter for the ue in dict_retr and current slot
def backoff(dict_retr, dict_backoffcnt, slot_list, slot):
    for i in slot_list[slot]:
        if i in dict_retr:
#             retrans_slot = int(slot + np.ceil((T_RAR + W_RAR + random.uniform(0, W_BO)) / T_RAREP))
#             dict_backoffcnt[i]= retrans_slot - slot
            dict_backoffcnt[i] = int(random.uniform(0,UORA_BO[dict_retr[i]]+1)) ## UORA 

# 0.a Simple Test

In [7]:
raslot = 14

In [8]:
ith_slot = 16

In [9]:
# Reset csv
h = pandas.DataFrame(columns = ["M","R","Ms", "Ps","Da","Imax","Actual Imax","Lmax","OCWmin","OCWmax"])
h.to_csv("simpletest_c_sim_R9T30_T16_T16_da.csv", index=False)

In [10]:
# Config
CONFIG='UORA'
starttime = datetime.datetime.now()
print('starttime:',starttime)
# h = pandas.DataFrame(columns = ["M","R","Ms", "Ps","Da","Imax","Actual Imax","Lmax","OCWmin","OCWmax"])    
# h.to_csv("simpletest_c_sim_RTTTT.csv", index=False)

# latency 
TXOP = 5.681
K = 1 # UORA
R = 9
sample = 1000
# number of max retransmission
retrans = 3 # N_PTmax - 1
# number of STA
NumMa=[10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200]
# OCWmin OCWmax
NumM = 200
UORA_BO = [0 for _ in range(0,retrans+2) ]
# setting of Imax
Imax_set = 16
# setting of OCWmin/max
OCWmin = 31
OCWmax = 31
# setting OCWn
UORA_BO[1]=OCWmin
for n in range (2, retrans+2):
    temp_OCW=2*UORA_BO[n-1]+1
    if temp_OCW<=OCWmax:
        UORA_BO[n]=temp_OCW
    else:
        UORA_BO[n]=UORA_BO[n-1]
UORA_BO.pop(0)

random.seed( 7 )

# Data
columns = ["M","R","Ms", "Ps","Da","Imax","Actual Imax","Lmax","OCWmin","OCWmax"]
Ps_list_N = []
MS_list_N = []
Da_list_A = []
Tp_list_A =[]
UE_list_N = []


# print(UORA_BO)
for i in range(20):
    avg_UE_N, MS_N_res, Ps_N_res,Da_A_res,Tp_A_res,Imax_A_res = main(sample,R,0)
    print(avg_UE_N, MS_N_res,Ps_N_res,Da_A_res,Tp_A_res,Imax_A_res)
    NumMa.pop(0)
    UE_list_N.append(avg_UE_N)    
    MS_list_N.append(MS_N_res)
    Ps_list_N.append(Ps_N_res)
    Da_list_A.append(Da_A_res)  ## UORA
    Tp_list_A.append(Tp_A_res)  ## UORA

print()
RU_list   = [R     for i in range(20)] 
Lmax_list  = [retrans+1     for i in range(20)] 
OCWmin_list = [OCWmin     for i in range(20)] 
OCWmax_list = [OCWmax     for i in range(20)] 
Imax_list = [Imax_set for i in range(20)]
Imax_list_actual = [raslot for i in range(20)]
data = {"M": UE_list_N  , 
        "R": RU_list     ,
        "Ms": MS_list_N  ,
        "Ps": Ps_list_N  ,
        "Da": Da_list_A  ,
        "Imax": Imax_list,
        "Actual Imax": Imax_list_actual,
        "Lmax": Lmax_list,
        "OCWmin": OCWmin_list,
        "OCWmax": OCWmax_list }


table = pandas.DataFrame(data)
cvs_pandas = table.to_csv("simpletest_c_sim_R9T30_T16_T16_da.csv", mode='a', index=False, header=False, columns=columns) 
endtime = datetime.datetime.now()
print('endtime:',endtime)
print('total:',endtime - starttime)

starttime: 2023-07-13 18:38:08.771809
Andy calculated da for grouping
10 9.997 0.9997 1073.6521899999998 0.9088181818181819 11
Andy calculated da for grouping
20 19.853 0.99265 2237.4220830000004 1.6544166666666669 12
Andy calculated da for grouping
30 28.219 0.940633 3356.9710719999994 2.170692307692308 13
Andy calculated da for grouping
40 32.141 0.803525 4001.4918839999996 2.472384615384615 13
Andy calculated da for grouping
50 30.915 0.6183 3983.1649780000002 2.378076923076923 13
Andy calculated da for grouping
60 27.302 0.455033 3624.6768349999993 2.1001538461538463 13
Andy calculated da for grouping
70 23.772 0.3396 3235.806704000001 1.8286153846153845 13
Andy calculated da for grouping
80 20.577 0.257212 2872.2113420000005 1.582846153846154 13
Andy calculated da for grouping
90 18.165 0.201833 2586.735411 1.3973076923076921 13
Andy calculated da for grouping
100 16.161 0.16161 2358.7512 1.2431538461538463 13
Andy calculated da for grouping
110 14.823 0.134755 2203.444022 1.14023

### Run all combination for different R (Imax = actual Imax)

In [11]:
def all_combination(RA_RU,Lmax_list, min_list, max_list):
    OCWmin =  min_list #[7,15,31,63,127]
    OCWmax =  max_list #[7,15,31,63,127]
    Lmax = Lmax_list #[1,2,3,4,5]
    R = RA_RU
    temp_min = 0
    temp_max = 0
    temp_Lmax = 0
    list = []

    for i in Lmax:
        for x in OCWmin:
            for y in OCWmax:
                OCW = [0 for _ in range(0,i+1) ]
                OCW[1] = x
                for n in range (2, i+1):
                    temp_OCW=2*OCW[n-1]+1
                    if temp_OCW<=y:
                        OCW[n]=temp_OCW
                    else:
                        OCW[n]=OCW[n-1]


                if (not((temp_Lmax == i)&(temp_max == OCW[i])&(temp_min == OCW[1]))):
                    T = 0
                    for z in range(1,i+1):
                        T = T + math.ceil(OCW[z]/R)
                    #print("Lmax: ",i,"\tOCWmin: ",OCW[1],"\tOCWmax: ",OCW[i], "\tT/Imax: ", T)
                    list.append([i,OCW[1],OCW[i],T])
                temp_max = OCW[i]
                temp_min = OCW[1]
                temp_Lmax = i
    list.sort(key = lambda s: s[3])
    return list

In [12]:
# 給定R來看有哪些組合(只留下極端組合 (Lmax最高 && OCWmax最小)和(Lmax最小 && OCWmax最大))
R = 18
list_main = []
list_main = all_combination(R,[1,2,3,4,5], [7,15,31,63,127], [7,15,31,63,127])
list1 = []



# 挑出有3組以上(Lmax, OCWmin, OCWmax)組合的T/Imax
data = np.array(list_main)
x = 1
for x in range(1,100):      
    data1= np.array(np.where(data[:,3] == x)).flatten()
    if data1.size>=3:
        list1.append(data1.tolist())
#print(list1)


# 留下(Lmax最高 && OCWmax最小)
temp_maxLmax = 1
temp_minOCWmax = 127
temp_nmax = 0
list2 = []
for i in range(len(list1)):
    for n in list1[i]:
        if list_main[n][0] >= temp_maxLmax and list_main[n][2] <= temp_minOCWmax:
            temp_maxLmax = list_main[n][0]
            temp_minOCWmax = list_main[n][2]
            temp_nmax = n
        else:
            continue
    list2.append(temp_nmax)
    temp_maxLmax = 1
    temp_minOCWmax = 127
    temp_nmax = 0
# print(list2)

# 留下(Lmax最小 && OCWmax最大)
temp_minLmax = 5
temp_maxOCWmax = 7
temp_nmin = 0
list3 = []
for i in range(len(list1)):
    for n in list1[i]:
        if list_main[n][0] <= temp_minLmax and list_main[n][2] >= temp_maxOCWmax:
            temp_minLmax = list_main[n][0]
            temp_maxOCWmax = list_main[n][2]
            temp_nmin = n
        else:
            continue
    list3.append(temp_nmin)
    temp_minLmax = 5
    temp_maxOCWmax = 7
    temp_nmin = 0
# print(list3)


# 更新list，3組以上的只留下極端組合，每種R Imax只會剩下1或2組
list1 = [b for a in list1 for b in a]
for i in range(len(list2)):
    list1.remove(list2[i])
for i in range(len(list3)):
    list1.remove(list3[i]) 
# print(list1)
list_main = [list_main[i] for i in range(len(list_main)) if i not in list1]

# 刪除例外狀況: 過濾成兩組以下後有相同Ps表現的需合併成一組(把Lmax相同且OCWn都小於R導致結果都相同的組合只留一個)
if list_main[0][0] == list_main[1][0] ==1 and list_main[0][2] < R and list_main[1][2] < R:
    list_main.pop(0)

print("[Lmax, OCWmin, OCWmax, Imax]")
for i in range(len(list_main)):
    print(list_main[i])    

[Lmax, OCWmin, OCWmax, Imax]
[1, 15, 15, 1]
[1, 31, 31, 2]
[2, 7, 7, 2]
[2, 15, 31, 3]
[3, 7, 7, 3]
[1, 63, 63, 4]
[4, 7, 7, 4]
[3, 15, 31, 5]
[5, 7, 7, 5]
[2, 31, 63, 6]
[4, 7, 31, 6]
[3, 15, 63, 7]
[4, 15, 31, 7]
[1, 127, 127, 8]
[5, 7, 31, 8]
[5, 15, 31, 9]
[3, 31, 63, 10]
[5, 31, 31, 10]
[4, 15, 63, 11]
[2, 63, 127, 12]
[5, 7, 63, 12]
[3, 31, 127, 14]
[4, 31, 63, 14]
[4, 15, 127, 15]
[5, 15, 63, 15]
[2, 127, 127, 16]
[4, 63, 63, 16]
[5, 31, 63, 18]
[3, 63, 127, 20]
[5, 63, 63, 20]
[4, 31, 127, 22]
[5, 15, 127, 23]
[3, 127, 127, 24]
[4, 63, 127, 28]
[5, 31, 127, 30]
[4, 127, 127, 32]
[5, 63, 127, 36]
[5, 127, 127, 40]


In [ ]:
# Run all combination for different R (Imax != actual Imax) 例如G = 2 下 因為觀察時間T的緣故 T2不能打滿

starttime = datetime.datetime.now()
print('starttime:',starttime)

h = pandas.DataFrame(columns = ["M","R","Ms", "Ps","Da","Imax","Actual Imax","Lmax","OCWmin","OCWmax"])    
h.to_csv("simpletest_c_sim_RT.csv", index=False)

# Config and sample setting
CONFIG='UORA'
## latency 
TXOP = 5.681
K = 1 # UORA
sample = 100000

for i in range(len(list_main)): 
    # setting of Imax
    Imax_set = list_main[i][3]
    # number of max retransmission
    retrans = list_main[i][0]-1 # N_PTmax - 1
    # setting of OCWmin/max
    OCWmin = list_main[i][1]
    OCWmax = list_main[i][2]    
    
    if Imax_set <=3:

        for j in range(1,Imax_set+1):
            raslot = j
            # number of STA
            NumMa=[10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200]
            # OCWmin OCWmax
            NumM = 200
            UORA_BO = [0 for _ in range(0,retrans+2) ]



            print(list_main[i][0],OCWmin,OCWmax, Imax_set, j)

            # setting OCWn
            UORA_BO[1]=OCWmin
            for n in range (2, retrans+2):
                temp_OCW=2*UORA_BO[n-1]+1
                if temp_OCW<=OCWmax:
                    UORA_BO[n]=temp_OCW
                else:
                    UORA_BO[n]=UORA_BO[n-1]
            UORA_BO.pop(0)

            print(UORA_BO)

            random.seed( 7 )

            # Data
            columns = ["M","R","Ms", "Ps","Da","Imax","Actual Imax","Lmax","OCWmin","OCWmax"] #
            Ps_list_N = []
            MS_list_N = []
            Da_list_A = []
            Tp_list_A =[]
            UE_list_N = []
            Imax_A_list = [] # UORA

            RU_list = []
            Lmax_list = []
            OCWmin_list = []
            OCWmax_list = []
            Imax_list = []
            Imax_list_actual=[]


            # print(UORA_BO)
            for i in range(20):
                avg_UE_N, MS_N_res, Ps_N_res,Da_A_res,Tp_A_res,Imax_A_res = main(sample,R,0)
                print(avg_UE_N, MS_N_res,Ps_N_res,Da_A_res,Tp_A_res,Imax_A_res)
                NumMa.pop(0)

                UE_list_N.append(avg_UE_N)    
                MS_list_N.append(MS_N_res)
                Ps_list_N.append(Ps_N_res)
                Da_list_A.append(Da_A_res)  ## UORA
                Tp_list_A.append(Tp_A_res)  ## UORA
                Imax_A_list.append(Imax_A_res)  ## UORA

            print()
            RU_list   = [R     for i in range(20)] 
            Lmax_list  = [retrans+1     for i in range(20)] 
            OCWmin_list = [OCWmin     for i in range(20)] 
            OCWmax_list = [OCWmax     for i in range(20)] 
            Imax_list = [Imax_set for i in range(20)]
            Imax_list_actual = [raslot for i in range(20)]
            data = {"M": UE_list_N  , 
                    "R": RU_list     ,
                    "Ms": MS_list_N  ,
                    "Ps": Ps_list_N  ,
                    "Da": Da_list_A  ,
                    "Imax": Imax_list,
                    "Actual Imax": Imax_list_actual,
                    "Lmax": Lmax_list,
                    "OCWmin": OCWmin_list,
                    "OCWmax": OCWmax_list }

            # starttime = datetime.datetime.now()
            # print('starttime:',starttime)
            table = pandas.DataFrame(data)
            cvs_pandas = table.to_csv("simpletest_c_sim_RT.csv", mode='a', index=False, header=False, columns=columns)        
        
    else:    
        for j in range(Imax_set - 3,Imax_set+1):
            raslot = j
            # number of STA
            NumMa=[10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200]
            # OCWmin OCWmax
            NumM = 200
            UORA_BO = [0 for _ in range(0,retrans+2) ]



            print(list_main[i][0],OCWmin,OCWmax, Imax_set, j)

            # setting OCWn
            UORA_BO[1]=OCWmin
            for n in range (2, retrans+2):
                temp_OCW=2*UORA_BO[n-1]+1
                if temp_OCW<=OCWmax:
                    UORA_BO[n]=temp_OCW
                else:
                    UORA_BO[n]=UORA_BO[n-1]
            UORA_BO.pop(0)

            print(UORA_BO)

            random.seed( 7 )

            # Data
            columns = ["M","R","Ms", "Ps","Da","Imax","Actual Imax","Lmax","OCWmin","OCWmax"] #
            Ps_list_N = []
            MS_list_N = []
            Da_list_A = []
            Tp_list_A =[]
            UE_list_N = []
            Imax_A_list = [] # UORA

            RU_list = []
            Lmax_list = []
            OCWmin_list = []
            OCWmax_list = []
            Imax_list = []
            Imax_list_actual=[]


            # print(UORA_BO)
            for i in range(20):
                avg_UE_N, MS_N_res, Ps_N_res,Da_A_res,Tp_A_res,Imax_A_res = main(sample,R,0)
                print(avg_UE_N, MS_N_res,Ps_N_res,Da_A_res,Tp_A_res,Imax_A_res)
                NumMa.pop(0)

                UE_list_N.append(avg_UE_N)    
                MS_list_N.append(MS_N_res)
                Ps_list_N.append(Ps_N_res)
                Da_list_A.append(Da_A_res)  ## UORA
                Tp_list_A.append(Tp_A_res)  ## UORA
                Imax_A_list.append(Imax_A_res)  ## UORA

            print()
            RU_list   = [R     for i in range(20)] 
            Lmax_list  = [retrans+1     for i in range(20)] 
            OCWmin_list = [OCWmin     for i in range(20)] 
            OCWmax_list = [OCWmax     for i in range(20)] 
            Imax_list = [Imax_set for i in range(20)]
            Imax_list_actual = [raslot for i in range(20)]
            data = {"M": UE_list_N  , 
                    "R": RU_list     ,
                    "Ms": MS_list_N  ,
                    "Ps": Ps_list_N  ,
                    "Da": Da_list_A  ,
                    "Imax": Imax_list,
                    "Actual Imax": Imax_list_actual,
                    "Lmax": Lmax_list,
                    "OCWmin": OCWmin_list,
                    "OCWmax": OCWmax_list }

            # starttime = datetime.datetime.now()
            # print('starttime:',starttime)
            table = pandas.DataFrame(data)
            cvs_pandas = table.to_csv("simpletest_c_sim_RT.csv", mode='a', index=False, header=False, columns=columns)


endtime = datetime.datetime.now()
print('endtime:',endtime)
print('total:',endtime - starttime)

starttime: 2023-07-13 18:38:37.887604
1 15 15 1 1
[15]
Andy calculated da for grouping
10 5.97649 0.597649 577.19147473 0.0 0
Andy calculated da for grouping
20 6.75252 0.337626 652.13812404 0.0 0
Andy calculated da for grouping
30 5.71928 0.190643 552.35090456 0.0 0
Andy calculated da for grouping
40 4.30049 0.107512 415.32842273 0.0 0
Andy calculated da for grouping
50 3.02832 0.060566 292.46606064 0.0 0
Andy calculated da for grouping
60 2.05364 0.034227 198.33439028 0.0 0
Andy calculated da for grouping
70 1.35931 0.019419 131.27808187 0.0 0
Andy calculated da for grouping
80 0.87515 0.010939 84.51936155 0.0 0
Andy calculated da for grouping
90 0.5564 0.006182 53.7354428 0.0 0
Andy calculated da for grouping
100 0.3464 0.003464 33.4542728 0.0 0
Andy calculated da for grouping
110 0.21912 0.001992 21.16195224 0.0 0
Andy calculated da for grouping
120 0.13348 0.001112 12.89109796 0.0 0
Andy calculated da for grouping
130 0.08193 0.00063 7.912553610000001 0.0 0
Andy calculated da for 

Andy calculated da for grouping
130 0.08188 0.00063 7.907724759999999 0.0 0
Andy calculated da for grouping
140 0.04946 0.000353 4.77669842 0.0 0
Andy calculated da for grouping
150 0.0302 0.000201 2.9166254 0.0 0
Andy calculated da for grouping
160 0.0181 0.000113 1.7480437000000002 0.0 0
Andy calculated da for grouping
170 0.01094 6.4e-05 1.05655238 0.0 0
Andy calculated da for grouping
180 0.00651 3.6e-05 0.6287162700000001 0.0 0
Andy calculated da for grouping
190 0.00406 2.1e-05 0.39210262 0.0 0
Andy calculated da for grouping
200 0.00211 1.1e-05 0.20377747 0.0 0

2 15 31 3 2
[15, 31]
Andy calculated da for grouping
10 8.08136 0.808136 792.37082535 8.08136 1
Andy calculated da for grouping
20 11.91515 0.595758 1180.08044167 0.0 0
Andy calculated da for grouping
30 12.29298 0.409766 1224.62698059 0.0 0
Andy calculated da for grouping
40 10.91132 0.272783 1091.3091924799999 0.0 0
Andy calculated da for grouping
50 9.00726 0.180145 903.80648085 0.0 0
Andy calculated da for grouping
6

In [ ]:
# Run all combination for different R (Imax = actual Imax)

starttime = datetime.datetime.now()
print('starttime:',starttime)

h = pandas.DataFrame(columns = ["M","R","Ms", "Ps","Da","Imax","Actual Imax","Lmax","OCWmin","OCWmax"])    
h.to_csv("simpletest_c_sim_RT.csv", index=False)

# Config and sample setting
CONFIG='UORA'
## latency 
TXOP = 5.681
K = 1 # UORA
sample = 10
raslot = 200
for i in range(len(list_main)):

    # number of max retransmission
    retrans = list_main[i][0]-1 # N_PTmax - 1
    # number of STA
    NumMa=[10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200]
    # OCWmin OCWmax
    NumM = 200
    UORA_BO = [0 for _ in range(0,retrans+2) ]
    # setting of Imax
    Imax_set = list_main[i][3]
    # setting of OCWmin/max
    OCWmin = list_main[i][1]
    OCWmax = list_main[i][2]
    # setting OCWn
    UORA_BO[1]=OCWmin
    for n in range (2, retrans+2):
        temp_OCW=2*UORA_BO[n-1]+1
        if temp_OCW<=OCWmax:
            UORA_BO[n]=temp_OCW
        else:
            UORA_BO[n]=UORA_BO[n-1]
    UORA_BO.pop(0)

    random.seed( 7 )

    # Data
    columns = ["M","R","Ms", "Ps","Da","Imax","Actual Imax","Lmax","OCWmin","OCWmax"] #
    Ps_list_N = []
    MS_list_N = []
    Da_list_A = []
    Tp_list_A =[]
    UE_list_N = []
    Imax_A_list = [] # UORA


    # print(UORA_BO)
    for i in range(20):
        avg_UE_N, MS_N_res, Ps_N_res,Da_A_res,Tp_A_res,Imax_A_res = main(sample,R,0)
        print(avg_UE_N, MS_N_res,Ps_N_res,Da_A_res,Tp_A_res,Imax_A_res)
        NumMa.pop(0)

        UE_list_N.append(avg_UE_N)    
        MS_list_N.append(MS_N_res)
        Ps_list_N.append(Ps_N_res)
        Da_list_A.append(Da_A_res)  ## UORA
        Tp_list_A.append(Tp_A_res)  ## UORA
        Imax_A_list.append(Imax_A_res)  ## UORA

    print()
    RU_list   = [R     for i in range(20)] 
    Lmax_list  = [retrans+1     for i in range(20)] 
    OCWmin_list = [OCWmin     for i in range(20)] 
    OCWmax_list = [OCWmax     for i in range(20)] 
    Imax_list = [Imax_set for i in range(20)]
    data = {"M": UE_list_N  , 
            "R": RU_list     ,
            "Ms": MS_list_N  ,
            "Ps": Ps_list_N  ,
            "Da": Da_list_A  ,
            "Imax": Imax_list,
            "Actual Imax": Imax_A_list,
            "Lmax": Lmax_list,
            "OCWmin": OCWmin_list,
            "OCWmax": OCWmax_list }

    # starttime = datetime.datetime.now()
    # print('starttime:',starttime)
    table = pandas.DataFrame(data)
    cvs_pandas = table.to_csv("simpletest_c_sim_RT.csv", mode='a', index=False, header=False, columns=columns)


endtime = datetime.datetime.now()
print('endtime:',endtime)
print('total:',endtime - starttime)


In [ ]:
# Config
CONFIG='UORA'
# number of max retransmission
retrans = 4 # N_PTmax - 1
# latency 
TXOP = 5.681
# backoff
UORA_BO =  [7,15,31,31,31] #[7,15,31,31,31,31,31,31]#OCWmin~max


NumM = 100 # 總數UE 也就是 M
# NumMa=[10]#, 20 ,30 ,40, 50, 60, 70, 80, 90]
numMa = 0
K = 1 # UORA

# initail number of EDT UE(Ma)
NumMa=[]     #=[50]
NumMa_list=[]# =[50]
for i in range(1, 10):
#     num_UE_EDT.append(i * 10)
#     num_UE_EDT_list.append(i * 10)
    NumMa.append(i * 10) #
    NumMa_list.append(i * 10)
    i += 10
# num_UE_EDT = [50] # 用NumMa取代了
# num_UE_EDT_list = [50]

# RU
Rutotal= 37 # 72
RuAs=[]
for i in range(1, Rutotal): # 當ruAfix為Rutotal-1時，有slot超出矩陣問題
    RuAs.append(i * 1)
    i+=1
# RuAs=[30] #36


# num_UE_EDT = [30]
# num_UE_EDT_list = [30]
# Assume All the UE support EDT Procedure
num_UE = [10]

ite = 1000
lock = mp.Lock()
# TBS = [328]



if __name__=='__main__':
    h = pandas.DataFrame(columns =  ["M_A", "R_A","K","MsA", "PsA","DaA","TpA","ImaxA"])    
    h.to_csv("simpletest_c_sim_k1.csv", index=False)
    starttime = datetime.datetime.now()
    print('starttime:',starttime)
    test_process(ite,36,1,lock)
    
    endtime = datetime.datetime.now()
    print('endtime:',endtime)
    print('total:',endtime - starttime)





In [ ]:
test5 = pandas.read_csv('simpletest5.csv')
test5

In [ ]:

simple_test = pandas.read_csv('simpletest3.csv')
simple_test

In [ ]:
# simple_test2 = pandas.read_csv('simpletest4.csv')

# h = pandas.DataFrame(columns = columns = ["M_T", "M_A", "M_N", "R_T", "R_A", "R_N","K", "MsT", "MsA", "MsN", "PsT", "PsA", "PsN"  
#                ,"DaT","DaA","DaN","TpT","TpA","TpN"]
# h.to_csv("simpletest4.csv", index=False)

# table = pandas.DataFrame(simple_test2)
#     lock.acquire()
#     cvs_pandas = table.to_csv("simpletest4.csv", mode='a', index=False, header=False, columns=columns)
#     lock.release()

simple_test2 = pandas.read_csv('simpletest4.csv')          
simple_test2


In [ ]:
MA_UEtest=[[-1, 6, 13, 36, 44, 47, -1, 0], [-1, 0, 17, 19, 58, 66, -1, 0], [-1, 2, 3, 33, 67, 92, -1, 1], [-1, 0, 12, 40, 74, 111, -1, 1], [-1, 0, 17, 46, 56, 103, -1, 0], [-1, 3, 13, 22, 33, 38, -1, 0], [-1, 4, 8, 9, 21, 22, -1, 0], [-1, 6, 7, 35, 40, 76, -1, 1], [-1, 0, 2, 12, 18, 41, -1, 1], [-1, 1, 2, 10, 32, 57, -1, 0], [-1, 0, 8, 11, 48, 62, -1, 1], [-1, 1, 5, 20, 44, 71, -1, 0], [-1, 2, 17, 19, 32, 52, -1, 1], [-1, 5, 10, 21, 22, 31, -1, 0], [-1, 0, 9, 27, 32, 37, -1, 0], [-1, 0, 8, 23, 29, 56, -1, 0], [-1, 2, 12, 46, 85, -1, -1, 1], [-1, 1, 4, 37, 60, 80, -1, 0], [-1, 0, 1, 27, 31, 36, -1, 0], [-1, 0, 6, 21, 23, 39, -1, 0], [-1, 4, 6, 37, 39, 60, -1, 0], [-1, 5, 11, 23, 58, 97, -1, 1], [-1, 0, 5, 34, 57, 61, -1, 0], [-1, 5, 7, 21, 56, 61, -1, 0], [-1, 6, 9, 33, 34, 68, -1, 1], [-1, 3, 17, 21, 57, 71, -1, 0], [-1, 0, 7, 29, 38, 39, -1, 0], [-1, 0, 11, 48, 49, -1, -1, 1], [-1, 0, 10, 40, 61, 114, -1, 1], [-1, 5, 7, 31, 66, 75, -1, 0], [-1, 5, 6, 38, 55, 75, -1, 0], [-1, 6, 7, 21, 48, 80, -1, 0], [-1, 6, 18, 22, 46, 73, -1, 0], [-1, 2, 17, 36, 58, 75, -1, 0], [-1, 1, 6, 28, 32, 55, -1, 0], [-1, 3, 5, 31, 61, 66, -1, 0], [-1, 1, 13, 34, 73, 100, -1, 1], [-1, 0, 17, 48, 74, 89, -1, 1], [-1, 5, 19, 24, -1, -1, -1, 1], [-1, 0, 6, 19, 20, 33, -1, 0], [-1, 3, 8, 17, 27, 36, -1, 0], [-1, 0, 8, 33, 59, 103, -1, 0], [-1, 3, 14, -1, -1, -1, -1, 1], [-1, 2, 7, 10, 21, 33, -1, 0], [-1, 5, 20, 59, 107, -1, -1, 1], [-1, 3, 4, 28, 47, 67, -1, 0], [-1, 2, 6, 38, 66, 103, -1, 0], [-1, 5, 6, 13, 44, 81, -1, 1], [-1, 1, 9, 28, 35, 58, -1, 0], [-1, 0, 12, 48, 75, 80, -1, 0]]

test_col=['arrive slot','1 apt','2 apt','3 apt','4 apt','5 apt','6','status']
test_index=[i for i in range(1,51)]
test_table = pandas.DataFrame(data=MA_UEtest[0::],columns=test_col,index= test_index)
test_table




In [ ]:
test_slotlist=[[1, 3, 4, 8, 10, 14, 15, 18, 19, 22, 26, 27, 28, 37, 39, 41, 49], [9, 11, 17, 18, 34, 36, 48], [2, 8, 9, 12, 16, 33, 43, 46], [2, 5, 25, 35, 40, 42, 45], [6, 17, 20, 45], [11, 13, 21, 22, 23, 29, 30, 35, 38, 44, 47], [0, 7, 19, 20, 24, 30, 31, 32, 34, 39, 46, 47], [7, 23, 26, 29, 31, 43], [6, 10, 15, 40, 41], [6, 14, 24, 48], [9, 13, 28, 43], [10, 21, 27], [3, 8, 16, 49], [0, 5, 36, 47], [42], [], [], [1, 4, 12, 25, 33, 37, 40], [8, 32], [1, 12, 38, 39], [11, 39, 44], [6, 13, 19, 23, 25, 31, 43], [5, 6, 13, 32], [15, 19, 21], [38], [], [], [14, 18, 40], [34, 45, 48], [15, 26], [], [13, 18, 29, 35], [9, 12, 14, 34], [2, 5, 24, 39, 41, 43], [22, 24, 36], [7, 48], [0, 18, 33, 40], [14, 17, 20], [5, 26, 30, 46], [19, 20, 26], [3, 7, 28], [8], [], [], [0, 11, 47], [], [4, 16, 32], [0, 45], [10, 27, 31, 37, 49], [27], [], [], [12], [], [], [30, 34], [4, 15, 23], [9, 22, 25], [1, 21, 33, 48], [41, 44], [17, 20], [22, 23, 28, 35], [10], [], [], [], [1, 29, 35, 46], [2, 45], [24], [], [], [11, 25], [], [32, 36], [3, 37], [29, 30, 33, 49], [7], [], [], [], [17, 31, 49], [47], [], [], [], [16], [], [], [], [37], [], [], [2], [], [], [], [], [21], [], [], [36], [], [], [4, 41, 46], [], [], [], [44], [], [], [], [3], [], [], [28], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]

test_slotlist[89]

# 0.z Others

In [ ]:
# TO do 
# 移除N_RU及L -> datadelay 相關
# self.N_RU[slot] += self.L[success_ue]

Todo 20211005 , 把推導的公式改成py檔 done 

In [ ]:
num_UE_EDT = []
num_UE_EDT_list = []
for i in range(1, 10):
    num_UE_EDT.append(i * 10)
    num_UE_EDT_list.append(i * 10)
    i += 10

In [ ]:
num_UE_EDT

In [ ]:
num_UE_EDT_list

In [ ]:
NumMa

In [ ]:
numofsta=3
test_retrans=4
Test_UE_list = [ [-1] * (test_retrans + 4) for _ in range(0, numofsta)] # retrans +3 for Ra /retrans +4 for UORA
Test_UE_list[1][test_retrans + 3] = 10
Test_UE_list

In [ ]:
# # inherit the class of initializing RA parameter
# class init_EDTparameter(init_RAparameter):
#     def __init__(self, config, sample, N, N_e):
#         super().__init__(config, sample, N_e)
#         self.convRA = init_RAparameter("conv_RA", sample, N)
#         self.accumulate_numUE = 0
#         self.accumulate_numUE_EDT = 0
#         self.P_s_T = 0
#         self.MsT = 0
#         self.Uti_T = 0
#         self.DdT = 0
#     def grouping(self):
#         self.numUE_to_convRA = 0
#         self.numUE_to_EDT = 0
#         self.dict_L_EDT = gen_UE_L(num_UE_EDT[0], L)
#         self.numUE_to_convRA, self.numUE_to_EDT, self.convRA.L, self.L = compare_L_and_TBS(self.dict_L_EDT, num_UE_EDT[0])
#         self.accumulate_numUE += (self.numUE_to_convRA + num_UE[0])
#         self.accumulate_numUE_EDT += self.numUE_to_EDT
#     def define_RAparameter(self):
#         self.grouping()
#         self.nUE = self.numUE_to_EDT
#         super().define_RAparameter()
#         self.convRA.nUE = self.numUE_to_convRA + num_UE[0]
#         self.convRA.define_RAparameter()
#     def arrival_process(self, slot):
#         super().arrival_process(slot)
#         self.convRA.arrival_process(slot)
#     def departure_process(self, slot, countdown):
#         super().departure_process(slot, countdown)
#         if len(self.convRA.dict_retr) > 0:
#              self.convRA.departure_process(slot, countdown)
#     def calculate_statistic(self):
#         super().calculate_statistic()
#         self.convRA.calculate_statistic()
#     def get_MS(self):
#         super().get_MS()
#     def report(self):
#         # calculat the average ue number for EDT UE and non-EDT UE
#         self.nUE = self.accumulate_numUE_EDT / self.sample
#         self.convRA.nUE = self.accumulate_numUE / self.convRA.sample
#         self.report_EDT()
#         self.convRA.report()
#         self.calculate_total()
#     # calculate the Ps_E, Da_E, Dd_E, T_DATA_E
#     def report_EDT(self):
#         self.get_MS()
#         # Average T_DATA
#         if self.MS != 0:
#             self.T_DATA = round((self.T_DATA / self.MS) * 1000000) / 1000000
#             # Calculate T_DATA
#             self.T_DATA = get_T_DATA(self.T_DATA, 1)
#         else:
#             self.T_DATA = 0.0

#         # Sucesss Probability - EDT
#         if self.nUE != 0:
#             self.P_s = round((float(self.MS) / float(self.nUE)) * 1000000) / 1000000
#         else:
#             self.P_s = 0.0

#         # Utilization - EDT
#         # self.Uti = 0
#         # if self.subcarrier != 0:
#         #     self.Uti = round((float(self.MS) / float(maxslot() * self.subcarrier)) * 1000000) / 1000000
#         # else:
#         #     self.Uti = 0.0

#         # Access delay - EDT
#         for temp_i in range(0, maxslot()):
#             self.T[temp_i] = (temp_i) * T_RAREP + T_RAR + W_RAR + self.T_DATA
#             for temp_n in range (0, retrans + 1):
#                 # self.Da_MiSn += self.iteMiSn[temp_i][temp_n] * float(self.T[temp_i])
#                 self.Dd_MiSn += self.iteMiSn[temp_i][temp_n] * float(self.T[temp_i])
#         if self.nUE != 0 and self.MS != 0:
#             # self.Da = float(self.Da_MiSn) / float(self.MS)
#             self.Dd = float(self.Dd_MiSn) / float(self.MS)
#         else:
#             self.Dd = 0.0
#     # calcualte the Ps_T, MsT, DdT, Uti_T
#     def calculate_total(self):
#         # PsT
#         if self.nUE != 0 or self.convRA.nUE != 0:
#             self.P_s_T = round(((float(self.MS) + float(self.convRA.MS)) / (float(self.nUE) + float(self.convRA.nUE))) * 1000000) / 1000000
#         else:
#             self.P_s_T = 0.0
#         # MsT
#         self.MsT = float(self.MS) + float(self.convRA.MS)
#         # DdT
#         if (self.MS + self.convRA.MS) != 0:
#             self.DdT = round((self.Dd * float(self.MS) + self.convRA.Dd * float(self.convRA.MS)) / (float(self.MS) + float(self.convRA.MS)) * 1000000) / 1000000
#         else:
#             self.DdT = 0.0
#         # Uti
#         # if self.subcarrier != 0 or self.convRA.subcarrier != 0:
#         #     self.Uti_T = round(((float(self.MS) + float(self.convRA.MS)) / (float(maxslot() * (self.subcarrier + self.convRA.subcarrier)))) * 1000000) / 1000000
#         # else:
#         #     self.Uti_T = 0.0